Enrichment
==========

Dependencies
------------

In [8]:
%lsmagic

Available line magics:
%activity  %cd  %connect_info  %conversation  %dot  %download  %edit  %get  %help  %html  %include  %install  %install_magic  %javascript  %jigsaw  %kernel  %kx  %latex  %load  %ls  %lsmagic  %macro  %magic  %matplotlib  %parallel  %plot  %pmap  %px  %python  %reload_magics  %restart  %run  %scala  %scheme  %set  %shell

Available cell magics:
%%activity  %%brain  %%conversation  %%debug  %%dot  %%file  %%help  %%html  %%init_spark  %%javascript  %%kx  %%latex  %%macro  %%pipe  %%processing  %%px  %%python  %%scala  %%scheme  %%shell  %%show  %%time  %%tutor


In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0 --transitive
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Intitializing Scala interpreter ...

Spark Web UI available at http://39ca38b8bb7d:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1531318983031)
SparkSession available as 'spark'


<console>: 2: error: ';' expected but double literal found.

In [5]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir284361146908668336/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir284361146908668336/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir284361146908668336/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom
Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir284361146908668336/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir284361146908668336/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir284361146908668336/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


General functions
-----------------

In [6]:
import org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql.functions._

In [7]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [8]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [9]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [10]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [11]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("inferSchema", true)
    .option("header", header)
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [12]:
def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
  import org.apache.spark.ml.feature.VectorAssembler
  import org.apache.spark.ml.linalg.Vectors
  val assembler = new VectorAssembler()
    .setInputCols(columns.toArray)
    .setOutputCol(output)
  assembler.transform(dataFrame.na.fill(0.0, columns).na.fill(0.0)).select(output)
}

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


In [13]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()
import spark.implicits._

sparkContext = org.apache.spark.SparkContext@1d7899e1
spark = org.apache.spark.sql.SparkSession@24f4628b


Spark app-20180711141935-0000: Some(http://39ca38b8bb7d:4040)

In [14]:
def rename(dataFrame: DataFrame, renamings: Map[String, String]) =   {
      val newColumns = dataFrame.columns.map(c=> renamings.getOrElse(c, c))
      dataFrame.toDF(newColumns:_*)
    }
def renameFunction(renamings: Map[String, String]) = udf[String, String](c=> renamings.getOrElse(c, c))

rename: (dataFrame: org.apache.spark.sql.DataFrame, renamings: Map[String,String])org.apache.spark.sql.DataFrame
renameFunction: (renamings: Map[String,String])org.apache.spark.sql.expressions.UserDefinedFunction


In [15]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"
val indexesPath = "indexes"
val reactomePath = indexesPath + "/reactome"
val unirefPath = expressionsPath + "/" + "uniref90"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
indexesPath = indexes
reactomePath = indexes/reactome
unirefPath = expressions/uniref90


expressions/uniref90

LAGs comparisons
---------------

In [20]:
import org.bdgenomics.adam.rdd.ADAMContext._
val genage = sc.loadFasta(az("indexes/genage/mouse_fly_worm_genage.fasta"))

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 19, 10.0.1.11, executor 142): ExecutorLostFailure (executor 142 exited caused by one of the running tasks) Reason: Unable to create executor due to Exception thrown in awaitResult: 
Driver stacktrace:
StackTrace:   at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1590)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1589)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1589)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:831)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.ap